In [ ]:
# MXNet package
from mxnet import model
from mxnet.gluon import nn
from mxnet.gluon.model_zoo.vision import resnet50_v1, mobilenet1_0, mobilenet_v2_1_0

# Custom package
import sys
sys.path.append("..")
from quantize.convert import convert_model
from quantize.freeze import FreezeHelper

## Construct a origin net without quantized parameters

In [ ]:
net = mobilenet1_0(pretrained=False)
net.output = nn.Dense(20)

## Freeze   
1. Create a helper with the origin net and quantized parameter file      
2. With the help of helper, find what symbol to exclude and what to quantize offline
3. Freeze

In [ ]:
helper = FreezeHelper(net, "./models/cifar100_mobilenet_1_0_quant_relu6-013000.params", (1,3,224,224))

In [ ]:
helper.list_symbols(suffix="fwd")

In [ ]:
# exclude first conv and last fc
exclude = ["mobilenet0_conv0_fwd", "dense0_fwd"]    

In [ ]:
# quantize weight offline
offline = [s.name for s in helper.list_symbols(suffix=("weight"))]     

In [ ]:
qsym, qargs, auxes = helper.freeze(exclude, offline, quantize_input_offline=False)

In [ ]:
model.save_checkpoint("./tmp/frozen_quantized_model", 0, qsym, qargs, auxes)